In [6]:
from app import app, db, bcrypt
from models import User, Stock, Portfolio
from flask import jsonify, request
import yfinance as yf
import pandas as pd
import sqlite3

In [3]:
coalIndia = yf.Ticker('COALINDIA.NS')

In [4]:
coalIndia.history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-11-04 00:00:00+05:30,105.067929,124.528962,105.067929,123.680473,479716245,0.0,0.0
2010-11-08 00:00:00+05:30,127.020253,128.500593,118.968660,119.419983,46932779,0.0,0.0
2010-11-09 00:00:00+05:30,119.203363,120.376802,117.343916,117.723022,23741956,0.0,0.0
2010-11-10 00:00:00+05:30,117.488306,118.354843,115.556643,116.549553,21057129,0.0,0.0
2010-11-11 00:00:00+05:30,116.621795,121.658561,116.206580,118.625664,26548372,0.0,0.0
...,...,...,...,...,...,...,...
2025-01-10 00:00:00+05:30,371.399994,373.200012,364.700012,368.500000,4893721,0.0,0.0
2025-01-13 00:00:00+05:30,366.100006,370.399994,361.250000,362.899994,8661589,0.0,0.0
2025-01-14 00:00:00+05:30,365.000000,374.000000,363.000000,369.250000,5698113,0.0,0.0


In [9]:
stock_data = coalIndia.history(period="max")

In [7]:
# Connect to SQLite database
conn = sqlite3.connect('stock_data.db')
cursor = conn.cursor()

In [8]:
# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS stock_prices (
        date TEXT PRIMARY KEY,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER,
        dividend REAL,
        StockSplits REAL
    )
''')

In [10]:
# Convert DataFrame to a list of tuples for SQL insertion
stock_data_tuples = [tuple(x) for x in stock_data.to_numpy()]
dates = stock_data.index.tolist()

In [11]:
stock_data_tuples

[(np.float64(105.06792234930926),
  np.float64(124.5289545601136),
  np.float64(105.06792234930926),
  np.float64(123.68046569824219),
  np.float64(479716245.0),
  np.float64(0.0),
  np.float64(0.0)),
 (np.float64(127.02026066228393),
  np.float64(128.50060137307437),
  np.float64(118.96866782398182),
  np.float64(119.41999053955078),
  np.float64(46932779.0),
  np.float64(0.0),
  np.float64(0.0)),
 (np.float64(119.20337874486336),
  np.float64(120.3768180545193),
  np.float64(117.34393096559475),
  np.float64(117.72303771972656),
  np.float64(23741956.0),
  np.float64(0.0),
  np.float64(0.0)),
 (np.float64(117.48835234758278),
  np.float64(118.35488994386775),
  np.float64(115.55668850633661),
  np.float64(116.54959869384766),
  np.float64(21057129.0),
  np.float64(0.0),
  np.float64(0.0)),
 (np.float64(116.62180271348697),
  np.float64(121.65856918859265),
  np.float64(116.20658797256587),
  np.float64(118.62567138671875),
  np.float64(26548372.0),
  np.float64(0.0),
  np.float64(0.0

In [12]:
dates

[Timestamp('2010-11-04 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-08 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-09 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-10 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-11 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-12 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-15 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-16 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-18 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-19 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-22 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-23 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-24 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-25 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-26 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-29 00:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2010-11-30 00:00:00+0530', tz='Asia/Kolkata'

In [15]:
# Insert data into SQLite table
for i, row in enumerate(stock_data_tuples):
    cursor.execute('''
        INSERT OR REPLACE INTO stock_prices 
        VALUES (?, ?, ?, ?, ?, ?, ?,?)
    ''', (dates[i].strftime('%Y-%m-%d'),) + row)

In [16]:
# Commit changes and close the connection
conn.commit()
conn.close()

In [17]:
# read

import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('stock_data.db')

# Read data from SQLite into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM stock_prices", conn)

# Close the connection
conn.close()

# Display the first few rows of the DataFrame
print(df.head())

# Optionally, you can convert 'date' to datetime for easier manipulation
df['date'] = pd.to_datetime(df['date'])

# Print some basic information about the dataset
print(df.info())

         date        open        high         low       close     volume  \
0  2010-11-04  105.067922  124.528955  105.067922  123.680466  479716245   
1  2010-11-08  127.020261  128.500601  118.968668  119.419991   46932779   
2  2010-11-09  119.203379  120.376818  117.343931  117.723038   23741956   
3  2010-11-10  117.488352  118.354890  115.556689  116.549599   21057129   
4  2010-11-11  116.621803  121.658569  116.206588  118.625671   26548372   

   dividend  StockSplits  
0       0.0          0.0  
1       0.0          0.0  
2       0.0          0.0  
3       0.0          0.0  
4       0.0          0.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3501 entries, 0 to 3500
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         3501 non-null   datetime64[ns]
 1   open         3501 non-null   float64       
 2   high         3501 non-null   float64       
 3   low          3501 non-nul

In [23]:
multipleStocks = yf.Tickers('HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS RELIANCE.NS TCS.NS')

In [24]:
ms_stockprices = multipleStocks.history(period="max")

[*********************100%***********************]  5 of 5 completed


In [25]:
ms_stockprices

Price              Close                                                       \
Ticker     ASIANPAINT.NS  HDFCBANK.NS HINDUNILVR.NS  RELIANCE.NS       TCS.NS   
Date                                                                            
1996-01-01           NaN     2.331029     34.400291     3.614025          NaN   
1996-01-02           NaN     2.327118     34.767647     3.585920          NaN   
1996-01-03           NaN     2.334940     34.561714     3.613147          NaN   
1996-01-04           NaN     2.319296     34.564495     3.579773          NaN   
1996-01-05           NaN     2.315385     34.508827     3.555181          NaN   
...                  ...          ...           ...          ...          ...   
2025-01-13   2252.649902  1630.849976   2451.000000  1239.849976  4280.898438   
2025-01-14   2239.949951  1646.599976   2367.949951  1238.750000  4222.986328   
2025-01-15   2229.699951  1643.050049   2373.000000  1252.199951  4239.497070   
2025-01-16   2215.600098  1652.050049   2345.000000  1266.449951  4196.299805   
2025-01-17   2261.000000  1636.750000   2354.000000  1302.349976  4124.299805   

Price          Dividends                                               ...  \
Ticker     ASIANPAINT.NS HDFCBANK.NS HINDUNILVR.NS RELIANCE.NS TCS.NS  ...   
Date                                                                   ...   
1996-01-01           NaN         0.0           0.0         0.0    NaN  ...   
1996-01-02           NaN         0.0           0.0         0.0    NaN  ...   
1996-01-03           NaN         0.0           0.0         0.0    NaN  ...   
1996-01-04           NaN         0.0           0.0         0.0    NaN  ...   
1996-01-05           NaN         0.0           0.0         0.0    NaN  ...   
...                  ...         ...           ...         ...    ...  ...   
2025-01-13           0.0         0.0           0.0         0.0    0.0  ...   
2025-01-14           0.0         0.0           0.0         0.0    0.0  ...   
2025-01-15           0.0         0.0           0.0         0.0    0.0  ...   
2025-01-16           0.0         0.0           0.0         0.0    0.0  ...   
2025-01-17           0.0         0.0           0.0         0.0   10.0  ...   

Price       Stock Splits                                               \
Ticker     ASIANPAINT.NS HDFCBANK.NS HINDUNILVR.NS RELIANCE.NS TCS.NS   
Date                                                                    
1996-01-01           NaN         0.0           0.0         0.0    NaN   
1996-01-02           NaN         0.0           0.0         0.0    NaN   
1996-01-03           NaN         0.0           0.0         0.0    NaN   
1996-01-04           NaN         0.0           0.0         0.0    NaN   
1996-01-05           NaN         0.0           0.0         0.0    NaN   
...                  ...         ...           ...         ...    ...   
2025-01-13           0.0         0.0           0.0         0.0    0.0   
2025-01-14           0.0         0.0           0.0         0.0    0.0   
2025-01-15           0.0         0.0           0.0         0.0    0.0   
2025-01-16           0.0         0.0           0.0         0.0    0.0   
2025-01-17           0.0         0.0           0.0         0.0    0.0   

Price             Volume                                                    
Ticker     ASIANPAINT.NS HDFCBANK.NS HINDUNILVR.NS  RELIANCE.NS     TCS.NS  
Date                                                                        
1996-01-01           NaN      350000         11000  104121369.0        NaN  
1996-01-02           NaN      412000        203500  168743308.0        NaN  
1996-01-03           NaN      284000         58000  209323879.0        NaN  
1996-01-04           NaN      282000        111500  216900264.0        NaN  
1996-01-05           NaN      189000         39500  166708467.0        NaN  
...                  ...         ...           ...          ...        ...  
2025-01-13     1181928.0    10834533       2595013   137

In [26]:
ms_stockprices["Close"]

Ticker,ASIANPAINT.NS,HDFCBANK.NS,HINDUNILVR.NS,RELIANCE.NS,TCS.NS
Date,,,,,
1996-01-01,NaN,2.331029,34.400291,3.614025,NaN
1996-01-02,NaN,2.327118,34.767647,3.585920,NaN
1996-01-03,NaN,2.334940,34.561714,3.613147,NaN
1996-01-04,NaN,2.319296,34.564495,3.579773,NaN
1996-01-05,NaN,2.315385,34.508827,3.555181,NaN
...,...,...,...,...,...
2025-01-13,2252.649902,1630.849976,2451.000000,1239.849976,4280.898438
2025-01-14,2239.949951,1646.599976,2367.949951,1238.750000,4222.986328
2025-01-15,2229.699951,1643.050049,2373.000000,1252.199951,4239.497070
